In [191]:
import re
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import bicgstab
from scipy import sparse
from glob import glob
import matplotlib.patches as patches
import pandas as pd
import math
from scipy.misc import imread
#import bcolz
import time
import os
import cmath

In [192]:
def fully_connected_e_neighbour_graph(img, sigma, e):
    array = np.array(img)
    rowN = np.shape(array)[0]
    colN = np.shape(array)[1]
    result = np.zeros((rowN * colN, rowN * colN))
    for row in range(rowN):
        for col in range(colN):
            for rowDiff in range(-e, e + 1, 1):
                for colDiff in range(-e, e + 1, 1):
                    # Avoid self edges
                    if rowDiff == 0 and colDiff == 0:
                        continue
                    if(row + rowDiff < rowN and -1 < row + rowDiff and col + colDiff < colN and -1 < col + colDiff):
                        if(rowDiff * rowDiff + colDiff * colDiff <= e * e):
                            row2 = row + rowDiff
                            col2 = col + colDiff
                            weight = math.exp(- math.pow(img[row][col] - img[row2][col2], 2) / (2 * math.pow(sigma, 2)))
                            result[row * colN + col][row2 * colN + col2] = weight
    # Convert it to sparse matrix
    # TODO: Direct initialization as sparse matrix is better
    return sp.csr_matrix(result)

def laplacian(graph):
    # diag = np.diag(sum(graph))
    diag = sp.diags((sum(graph)).toarray()[0], 0)
    lap = diag - graph
    return [lap, diag]

def makeChessBoard(WhiteFrameWidth, chessNodeWidth, chessElementNumberPerRow):
    size = WhiteFrameWidth * 2 + chessElementNumberPerRow * chessNodeWidth
    A = np.zeros((size, size))
    for i in range(size - 2 * WhiteFrameWidth):
        for j in range(size - 2 * WhiteFrameWidth):
            if ((int(i / chessNodeWidth)) + (int(j / chessNodeWidth))) % 2 == 0:
                A[WhiteFrameWidth + i][WhiteFrameWidth + j] = 200
    return A

def starOpt_fast(graph, c, gamma, s, k, max_iter):
    n = graph.shape[0]
    graph = sp.csr_matrix(graph)
    
    [coords_i, coords_j, V] = sp.find(graph)
    [tri_i, tri_j, V] = sp.find(sp.triu(graph))
    
    mapping = triuToFullIdx(coords_i, coords_j, tri_i, tri_j, graph)
    
    degs = sp.csr_matrix(np.transpose(sp.csr_matrix.sum(graph, 0)))
    
    eta = 5;
    p = 2 * (np.dot(np.transpose(c), c)) / gamma;
    
    # TODO: Continue from here...
    
    return 1

def triuToFullIdx(coords_i, coords_j, tri_i, tri_j, graph):
    # Map each edge in graph to the one in sp.triu(graph)
    n = graph.shape[0]
    mapping = np.zeros((coords_i.shape[0], 1))
    triIdx = {}
    for i in range(len(tri_i)):
        triIdx[tri_i[i] * n + tri_j[i]] = i
    for i in range(len(coords_i)):
        if coords_i[i] < coords_j[i]:
            mapping[i] = triIdx[coords_i[i] * n + coords_j[i]]
        else:
            mapping[i] = triIdx[coords_j[i] * n + coords_i[i]]
    return mapping
        

In [193]:
A = np.array([[0, 2], [3, 0]])
sigma = 5
e = 2
graph = fully_connected_e_neighbour_graph(A, sigma, e)
[L, D] = laplacian(graph)
gamma = 1
s = 2000
k = 2
max_iter = 500
inverse = sp.linalg.inv(10 * L + sp.csr_matrix(np.identity(graph.shape[0])))
proj = np.random.normal(0,1,(graph.shape[0],2))
field = inverse.dot(proj)
c = (sp.linalg.inv(D)).dot(field)

n = graph.shape[0]
print(graph.toarray())
degs = (sp.csr_matrix(np.transpose(sp.csr_matrix.sum(graph, 0))))
eta = 5;
p = 2 * (np.dot(np.transpose(c), c)) / gamma;

y = np.random.normal(0,1,(n,k))

y = np.dot(y, pow(p / sp.csr_matrix.sum(np.sum(np.multiply(degs.dot(sp.csr_matrix(np.ones((1,k)))), np.multiply(y, y)))), .5))
print(y)
# y can have some imaginary numbers so it says nan...
# u_s = starOpt_fast(graph, c, gamma, s, k, max_iter)

[[0.         0.92311635 0.83527021 1.        ]
 [0.92311635 0.         0.98019867 0.92311635]
 [0.83527021 0.98019867 0.         0.83527021]
 [1.         0.92311635 0.83527021 0.        ]]
[[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]]


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:295: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:202: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  'is in the CSC matrix format', SparseEfficiencyWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:133: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  SparseEfficiencyWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in sqrt


In [194]:
A = np.array([[0, 2], [3, 0]])
sigma = 5
e = 2
k = 2
graph = fully_connected_e_neighbour_graph(A, sigma, e)
n = graph.shape[0]
print(graph.toarray())
degs = (sp.csr_matrix(np.transpose(sp.csr_matrix.sum(graph, 0))))
eta = 5;
p = 2 * (np.dot(np.transpose(c), c)) / gamma;
y = np.random.normal(0,1,(n,k))
print(p / sp.csr_matrix.sum(np.sum(np.multiply(degs.dot(sp.csr_matrix(np.ones((1,k)))), np.multiply(y, y)))))

[[0.         0.92311635 0.83527021 1.        ]
 [0.92311635 0.         0.98019867 0.92311635]
 [0.83527021 0.98019867 0.         0.83527021]
 [1.         0.92311635 0.83527021 0.        ]]
[[ 9.57007804e-05 -4.37672545e-05]
 [-4.37672545e-05  2.45604118e-05]]
